In [1]:
# Downloading necessary files
!git clone https://github.com/someilay/Signals_Assignment_2.git
!mv Signals_Assignment_2/barbie_vs_puppy barbie_vs_puppy
!mv Signals_Assignment_2/training training
!mv Signals_Assignment_2/validation validation
!rm -R Signals_Assignment_2

!wget https://raw.githubusercontent.com/someilay/Signals_Assignment_1/master/thinkdsp.py

--2023-02-26 18:42:00--  https://raw.githubusercontent.com/someilay/Signals_Assignment_1/master/thinkdsp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48687 (48K) [text/plain]
Saving to: ‘thinkdsp.py’

thinkdsp.py         100%[===================>]  47.55K  --.-KB/s    in 0.06s   

2023-02-26 18:42:00 (808 KB/s) - ‘thinkdsp.py’ saved [48687/48687]



In [482]:
# If you want to generate data from scratch
!rm -R training
!rm -R validation

# Preprocessing

In [491]:
import numpy as np
import matplotlib.pyplot as plt
import thinkdsp as tdp
import os
import shutil

from typing import Tuple, Optional, List
from thinkdsp import decorate
from thinkdsp import read_wave
from PIL import Image
from scipy.ndimage import gaussian_filter

np.set_printoptions(precision=3, suppress=True)

In [495]:
BARBIE, PUPPY = 'barbie', 'puppy'

BARBIE_PATH = 'barbie_vs_puppy/barbie/'
PUPPY_PATH = 'barbie_vs_puppy/puppy/'

BARBIE_SAVE_PATH = 'training/barbie/'
PUPPY_SAVE_PATH = 'training/puppy/'

V_BARBIE_SAVE_PATH = 'validation/barbie/'
V_PUPPY_SAVE_PATH = 'validation/puppy/'

SEG_LENGTH = 256
SPEC_STATS = dict()
MAX_FREQ = 16000
VAL_SIZE = 100

In [281]:
def get_files(_path: str):
    for _file in os.listdir(_path):
        if os.path.isfile(os.path.join(_path, _file)):
            yield _file

def min_max_duration(_data_path: str) -> Tuple[float, float, float]:
    _max_d = -1
    _min_d = np.inf
    _mean = 0
    _i = 0
    for _file in get_files(_data_path):
        _cur = read_wave(_data_path + _file)
        _mean += _cur.duration
        _i += 1
        if _cur.duration > _max_d:
            _max_d = _cur.duration
        if _cur.duration < _min_d:
            _min_d = _cur.duration
    return _min_d, _max_d, _mean / _i

for path in [BARBIE_PATH, PUPPY_PATH]:
    min_d, max_d, mean_d = min_max_duration(path)
    print(f'{path}; min dur.: {min_d:.2f} s; max dur.: {max_d:.2f} s; mean dur.: {mean_d:.2f} s;')

barbie_vs_puppy/barbie/; min dur.: 1.12 s; max dur.: 5.53 s; mean dur.: 2.21 s;
barbie_vs_puppy/puppy/; min dur.: 1.16 s; max dur.: 3.39 s; mean dur.: 1.96 s;


In [284]:
def spectrogram_stats(_data_path: str, _seg_length: int, _max_freq: float) -> List[float]:
    _min_h = np.inf
    _min_w = np.inf
    _max_h = -1
    _max_w = -1
    _mean_h = 0
    _mean_w = 0
    _i = 0

    for _file in get_files(_data_path):
        _i += 1
        _h, _w = read_wave(_data_path + _file)\
            .make_spectrogram(_seg_length)\
            .get_data(_max_freq)\
            .shape
        _min_h = min(_min_h, _h)
        _min_w = min(_min_w, _w)
        _max_h = max(_max_h, _h)
        _max_w = max(_max_w, _w)
        _mean_h += _h
        _mean_w += _w

    return [_min_h, _min_w, _max_h, _max_w, int(_mean_h / _i), int(_mean_w / _i)]

for path in [BARBIE_PATH, PUPPY_PATH]:
    stats = spectrogram_stats(path, SEG_LENGTH, MAX_FREQ)
    SPEC_STATS[path] = stats
    print(f'{path}; min height, min width, max h., max w., mean h. and mean w.: {stats}')

barbie_vs_puppy/barbie/; min height, min width, max h., max w., mean h. and mean w.: [85, 418, 85, 2070, 85, 828]
barbie_vs_puppy/puppy/; min height, min width, max h., max w., mean h. and mean w.: [85, 433, 93, 1270, 85, 730]


In [70]:
def roll_zero_pad(a: np.ndarray, shift: int):
    res = np.roll(a, shift)
    if shift > 0:
        res[:shift] = 0
    if shift < 0:
        res[shift:] = 0
    return res

In [105]:
def mul_wave(_wave: tdp.Wave, _val: float) -> tdp.Wave:
    return tdp.Wave(_wave.ys * _val, _wave.ts, _wave.framerate)

In [79]:
def padding(_wave: tdp.Wave, _shift: float) -> tdp.Wave:
    _shift_l = int(len(_wave.ys) * _shift)
    _ys = roll_zero_pad(_wave.ys, _shift_l)
    return tdp.Wave(_ys, _wave.ts, _wave.framerate)

In [295]:
def noice(_wave: tdp.Wave, _g_factor: float, _b_factor: float) -> tdp.Wave:
    _gaussian = tdp.UncorrelatedGaussianNoise()
    _brownian = tdp.BrownianNoise()

    _g_noice = _gaussian.make_wave(duration=_wave.duration, framerate=_wave.framerate)
    _g_noice = mul_wave(_g_noice, _g_factor)

    _b_noice = _brownian.make_wave(duration=_wave.duration, framerate=_wave.framerate)
    _b_noice = mul_wave(_b_noice, _b_factor)

    return _g_noice + _b_noice + _wave

In [418]:
def random_pad(vec, pad_width, _, kwargs):
    _factor = kwargs.get('factor', 1)
    vec[:pad_width[0]] = abs(np.random.normal(size=pad_width[0]) * _factor)
    vec[vec.size-pad_width[1]:] = abs(np.random.normal(size=pad_width[1]) * _factor)

def noice_pad(_arr: np.ndarray, _shift: float) -> np.ndarray:
    _h = int(_arr.shape[0] * _shift)
    _res = _arr

    if _h > 0:
        _factor = np.mean(_arr[:_h])
        _res = np.pad(_arr, ((_h, 0), (0, 0)), mode=random_pad, factor=_factor)[:-_h]
    if _h < 0:
        _factor = np.mean(_arr[_h:])
        _res = np.pad(_arr, ((0, -_h), (0, 0)), mode=random_pad, factor=_factor)[-_h:]

    return _res

In [483]:
import multiprocessing as mp

os.makedirs(BARBIE_SAVE_PATH, exist_ok=True)
os.makedirs(PUPPY_SAVE_PATH, exist_ok=True)

def augment_sample(_file_path: str, _save_path: str, _params: tuple) -> tdp.Wave:
    _padding, _g_factor, _b_factor, _n_padding = _params
    _wave = read_wave(_file_path)
    _wave = padding(_wave, _padding)
    _wave = noice(_wave, _g_factor, _b_factor)
    _sp = _wave.make_spectrogram(SEG_LENGTH).get_data(MAX_FREQ)
    _sp = noice_pad(_sp, _n_padding)
    _img = Image.fromarray(np.uint8(_sp * 255) , 'L')
    _img.save(_save_path, bitmap_format=_save_path[-3:])
    return _wave


def augment_data(_data_path: str, _save_path: str,
                 _class: str, _telemetry_step: int = 400,
                 _n_jobs: Optional[int] = None) -> tdp.Wave:
    _paddings = np.linspace(-0.1, 0.1, 5)
    _g_factors = np.linspace(0, 0.01, 4)
    _b_factors = np.linspace(0.1, 0.5, 5)
    _n_paddings = np.linspace(-0.05, 0.05, 3)
    _arguments = []
    _i = 0

    if _n_jobs is None:
        _n_jobs = mp.cpu_count() // 2

    for _file in get_files(_data_path):
        _file_path = _data_path + _file
        for _padding in _paddings:
            for _g_factor in _g_factors:
                for _b_factor in _b_factors:
                    for _n_padding in _n_paddings:
                        _save = _save_path + _class + f'_{_i}.png'
                        _arguments.append((_file_path, _save, (_padding, _g_factor, _b_factor, _n_padding)))
                        _i += 1

    _r_ind = np.random.randint(0, len(_arguments) - 1)
    _r_argument = _arguments[_r_ind]
    _arguments.pop(_r_ind)
    _r_sample = augment_sample(*_r_argument)
    _big_steps = len(_arguments) // _telemetry_step + int(len(_arguments) % _telemetry_step != 0)

    for _j in range(_big_steps):
        with mp.Pool(_n_jobs) as p:
            p.starmap(
                augment_sample,
                _arguments[_j * _telemetry_step: (_j + 1) * _telemetry_step]
            )
        _steps_c = min((_j + 1) * _telemetry_step, len(_arguments) + 1)
        print(f'{_steps_c} steps are completed!')

    return _r_sample

In [494]:
os.makedirs(V_BARBIE_SAVE_PATH, exist_ok=True)
os.makedirs(V_PUPPY_SAVE_PATH, exist_ok=True)

def gen_validation(_data_path: str, _val_path: str, _count: int):
    _files = [
        _file for _file in os.listdir(_data_path)
        if os.path.isfile(os.path.join(_data_path, _file))
    ]
    _val_set = np.random.choice(_files, _count)
    for _item in _val_set:
        shutil.move(_data_path + _item, _val_path + _item)

In [496]:
b_sample = augment_data(
    BARBIE_PATH, BARBIE_SAVE_PATH, BARBIE,
    _telemetry_step=20 * mp.cpu_count(),
    _n_jobs=mp.cpu_count()
)

240 steps are completed!
480 steps are completed!
720 steps are completed!
960 steps are completed!
1200 steps are completed!
1440 steps are completed!
1680 steps are completed!
1920 steps are completed!
2160 steps are completed!
2400 steps are completed!
2640 steps are completed!
2880 steps are completed!
3120 steps are completed!
3360 steps are completed!
3600 steps are completed!
3840 steps are completed!
4080 steps are completed!
4320 steps are completed!
4560 steps are completed!
4800 steps are completed!
5040 steps are completed!
5280 steps are completed!
5520 steps are completed!
5760 steps are completed!
6000 steps are completed!
6240 steps are completed!
6480 steps are completed!
6720 steps are completed!
6960 steps are completed!
7200 steps are completed!
7440 steps are completed!
7680 steps are completed!
7920 steps are completed!
8160 steps are completed!
8400 steps are completed!
8640 steps are completed!
8880 steps are completed!
9120 steps are completed!
9360 steps are c

In [497]:
p_sample = augment_data(
    PUPPY_PATH, PUPPY_SAVE_PATH, PUPPY,
    _telemetry_step=20 * mp.cpu_count(),
    _n_jobs=mp.cpu_count()
)

240 steps are completed!
480 steps are completed!
720 steps are completed!
960 steps are completed!
1200 steps are completed!
1440 steps are completed!
1680 steps are completed!
1920 steps are completed!
2160 steps are completed!
2400 steps are completed!
2640 steps are completed!
2880 steps are completed!
3120 steps are completed!
3360 steps are completed!
3600 steps are completed!
3840 steps are completed!
4080 steps are completed!
4320 steps are completed!
4560 steps are completed!
4800 steps are completed!
5040 steps are completed!
5280 steps are completed!
5520 steps are completed!
5760 steps are completed!
6000 steps are completed!
6240 steps are completed!
6480 steps are completed!
6720 steps are completed!
6960 steps are completed!
7200 steps are completed!
7440 steps are completed!
7680 steps are completed!
7920 steps are completed!
8160 steps are completed!
8400 steps are completed!
8640 steps are completed!
8880 steps are completed!
9120 steps are completed!
9360 steps are c

In [498]:
b_sample.make_audio()

In [499]:
p_sample.make_audio()

In [500]:
gen_validation(BARBIE_SAVE_PATH, V_BARBIE_SAVE_PATH, VAL_SIZE)
gen_validation(PUPPY_SAVE_PATH, V_PUPPY_SAVE_PATH, VAL_SIZE)

# Training

In [107]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

In [108]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = 'training/'
VALID_DATA_DIR = 'validation/'

In [109]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 216 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [110]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

In [111]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [112]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 109, 109, 16)      2320      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 54, 54, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 26, 26, 32)     

In [113]:
EPOCHS = 2
BATCH_SIZE = 16
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/2
6/6 [==============================] - 9s 1s/step - loss: 0.7139 - get_f1: 0.5955 - val_loss: 0.3067 - val_get_f1: 0.8437
Epoch 2/2
6/6 [==============================] - 5s 806ms/step - loss: 0.5922 - get_f1: 0.6806 - val_loss: 0.5302 - val_get_f1: 0.7500


In [114]:
model.save_weights('firstname_secondname')